In [1]:
!pip install wikipedia-api

In [2]:
import wikipediaapi

In [5]:
to_ignore = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Photo gallery",
    "Photos",
    "Gallery",
    "Works cited",
    "Notes",
    "Notes and references",
    "References and sources",
    "References and notes",
]

In [13]:
import time
import json
import os

In [9]:
def get_sections(
          section: wikipediaapi.WikipediaPageSection,
          parent_title: list[str],
          sections_to_ignore: set[str] = to_ignore,
      ) -> list[tuple[list[str],str]]:
 
  """Gather sections and subsections data."""

  sect_title = section.title 
  title = parent_title + [sect_title]
  title.extend([sect_title])
  results = []
  
  if sect_title not in sections_to_ignore:
    sect_text = section.text   
    string_section = ( title , sect_text)
    results.extend([string_section])
    if len(section.sections)>1:
      for subsection in section.sections:
        get_sections(subsection, title, sections_to_ignore)     
  return results

def get_pages (
            page: wikipediaapi.WikipediaPage,
            sections_to_ignore: set[str] = to_ignore,
        )-> list[tuple[list[str],str]]:

    """Gather the page information: title and summary, and then go deep in sections inforamtion."""

    parent_title = page.title
    print(f"parent_title = {parent_title} ")

    summary = page.summary
    string_parent = ([parent_title], summary)
    results=[string_parent]

    if len(page.sections) > 0:
      for section in page.sections:
        results.extend(get_sections(section,[parent_title]))

    return results

user_agent = "Data collection (patrickyamin@gmail.com) for ML Search and Retrieval"

wiki = wikipediaapi.Wikipedia(user_agent)

CATEGORY_TITLE = "Category:Featured articles"

# Get the category object for "Featured Articles"
cat = wiki.page(CATEGORY_TITLE)

# Print the name of the category
print("Name: ", cat.categorymembers)

# Get the articles included in this category
# # if it's an article:  wikipediaapi.Namespace.MAIN = 0
# # if it's a subcategory: wikipediaapi.Namespace.CATEGORY = 14
articles = [w for w in cat.categorymembers.values() if w.ns == wikipediaapi.Namespace.MAIN]

wiki_corpus=[]
for page in articles:
  wiki_corpus.extend(get_pages(page))
wiki_corpus[:5]

Name:  {'? (film)': ? (film) (id: ??, ns: 0), '0.999...': 0.999... (id: ??, ns: 0), '1 − 2 + 3 − 4 + ⋯': 1 − 2 + 3 − 4 + ⋯ (id: ??, ns: 0), '1 Line (Sound Transit)': 1 Line (Sound Transit) (id: ??, ns: 0), '1st Cavalry Division (Kingdom of Yugoslavia)': 1st Cavalry Division (Kingdom of Yugoslavia) (id: ??, ns: 0), '1st Missouri Field Battery': 1st Missouri Field Battery (id: ??, ns: 0), '1st Provisional Marine Brigade': 1st Provisional Marine Brigade (id: ??, ns: 0), '2nd Infantry Division (United Kingdom)': 2nd Infantry Division (United Kingdom) (id: ??, ns: 0), '2nd Red Banner Army': 2nd Red Banner Army (id: ??, ns: 0), '3 of Hearts (album)': 3 of Hearts (album) (id: ??, ns: 0), '4 (Beyoncé album)': 4 (Beyoncé album) (id: ??, ns: 0), '4 Minutes': 4 Minutes (id: ??, ns: 0), 'Coventry ring road': Coventry ring road (id: ??, ns: 0), '4th Army (Kingdom of Yugoslavia)': 4th Army (Kingdom of Yugoslavia) (id: ??, ns: 0), '4th Missouri Infantry Regiment (Confederate)': 4th Missouri Infantry 

[(['? (film)'],
  '? (also written Tanda Tanya, meaning Question Mark) is a 2011 Indonesian drama film directed by Hanung Bramantyo. It stars Revalina Sayuthi Temat, Reza Rahadian, Agus Kuncoro, Endhita, Rio Dewanto, and Hengky Sulaeman. The film focuses around Indonesia\'s religious pluralism, which often results in conflict between different beliefs, represented in a plot that revolves around the interactions of three families, one Buddhist, one Muslim, and one Catholic. After undergoing numerous hardships and the deaths of several family members in religious violence, they are reconciled.\nBased on Bramantyo\'s experiences as a mixed-race child, ? was meant to counter the portrayal of Islam as a "radical religion". Owing to the film\'s theme of religious pluralism and controversial subject matter, Bramantyo had difficulty finding backing. Eventually, Mahaka Pictures put forth Rp 5 billion ($600,000) to fund the production. Filming began on 5 January 2011 in Semarang.\nReleased on 7 

In [12]:
wiki_corpus

[(['? (film)'],
  '? (also written Tanda Tanya, meaning Question Mark) is a 2011 Indonesian drama film directed by Hanung Bramantyo. It stars Revalina Sayuthi Temat, Reza Rahadian, Agus Kuncoro, Endhita, Rio Dewanto, and Hengky Sulaeman. The film focuses around Indonesia\'s religious pluralism, which often results in conflict between different beliefs, represented in a plot that revolves around the interactions of three families, one Buddhist, one Muslim, and one Catholic. After undergoing numerous hardships and the deaths of several family members in religious violence, they are reconciled.\nBased on Bramantyo\'s experiences as a mixed-race child, ? was meant to counter the portrayal of Islam as a "radical religion". Owing to the film\'s theme of religious pluralism and controversial subject matter, Bramantyo had difficulty finding backing. Eventually, Mahaka Pictures put forth Rp 5 billion ($600,000) to fund the production. Filming began on 5 January 2011 in Semarang.\nReleased on 7 

In [16]:
def get_sections(section, parent_title, sections_to_ignore):
    """Gather sections and subsections data."""
    sect_title = section.title 
    title = parent_title + [sect_title]  # Create a new list to avoid modifying the parent title
    results = []
  
    if sect_title not in sections_to_ignore:
        sect_text = section.text
        string_section = (title, sect_text)
        results.append(string_section)  # Add the current section

        for subsection in section.sections:
            results += get_sections(subsection, title, sections_to_ignore)  # Accumulate subsection results

    return results

def get_pages(page, sections_to_ignore):
    """Gather the page information: title and summary, and then go deep in sections information."""
    parent_title = page.title
    summary = page.summary
    string_parent = ([parent_title], summary)
    results = [string_parent]

    if page.sections:
        for section in page.sections:
            results.extend(get_sections(section, [parent_title], sections_to_ignore))

    return results

def load_corpus(file_path):
    """Load the existing corpus from a file."""
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return json.load(file)
    return []

def save_corpus(file_path, corpus):
    """Save the corpus to a file."""
    with open(file_path, 'w') as file:
        json.dump(corpus, file)

# Wikipedia API setup
user_agent = "Data collection (patrickyamin@gmail.com) for ML Search and Retrieval"
wiki = wikipediaapi.Wikipedia(user_agent)

# Corpus file path
corpus_file_path = '../capstone/wiki_corpus.json'

# Load existing corpus
wiki_corpus = load_corpus(corpus_file_path)

# Define sections to ignore
to_ignore = {'References', 'External links', 'See also', 'Notes'}

# Get "Featured Articles" category
cat = wiki.page("Category:Featured articles")
articles = [w for w in cat.categorymembers.values() if w.ns == wikipediaapi.Namespace.MAIN]

# Process articles
for i, page in enumerate(articles):
    try:
        wiki_corpus.extend(get_pages(page, to_ignore))
        # Save progress every 100 articles
        if i % 100 == 0:
            save_corpus(corpus_file_path, wiki_corpus)
        time.sleep(1)  # Rate limiting
    except Exception as e:
        print(f"Error processing article {page.title}: {e}")

# Save final corpus
save_corpus(corpus_file_path, wiki_corpus)

print("Data collection complete.")

Error processing article Fort Ticonderoga: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Read timed out. (read timeout=10.0)
Data collection complete.


In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_json('wiki_corpus.zip')

In [3]:
df.shape

,0,1
0,[? (film)],"? (also written Tanda Tanya, meaning Question ..."
1,"[? (film), Plot]",? focuses on interfaith relations in Indonesia...
2,"[? (film), Cast]","Revalina S. Temat as Menuk, a religious Muslim..."
3,"[? (film), Production]","? was directed by Hanung Bramantyo, who is of ..."
4,"[? (film), Themes and style]","Ade Irwansyah, writing for Tabloid Bintang, no..."


In [4]:
df.shape

(90618, 2)